In [ ]:
import pandas as pd 
import cv2   
import numpy as np 
import os
import tensorflow as tf 
import matplotlib.pyplot as plt
import pathlib
import matplotlib.image as mpimg
from keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [ ]:
train_path="../input/self2/archive (1)/New Masks Dataset/Train"
train_data=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224),
                         classes=['Mask', 'Non Mask'], batch_size=32, shuffle= True)

val_path="../input/self2/archive (1)/New Masks Dataset/Validation"
val_data=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=val_path, target_size=(224,224),
                         classes=['Mask', 'Non Mask'], batch_size=32, shuffle = True)

test_path="../input/self2/archive (1)/New Masks Dataset/Test"
test_data=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224),
                         classes=['Mask', 'Non Mask'], batch_size=32, shuffle = False)

In [ ]:
train_data.class_indices

In [ ]:
test_data.class_indices

In [ ]:
test_data.classes

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224,224,3)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(256, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(512, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation="relu"),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(num_classes, activation="softmax")
])

model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
  train_data,
  validation_data=val_data,
  epochs=15
)

In [ ]:
model.evaluate(test_data)

In [ ]:
predictions = model.predict(x=test_data, steps=len(test_data))

In [ ]:
#print(predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=test_data.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['Mask','Non Mask']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
path='../input/face-mask-detection/dataset/with_mask/image_113.png'
im=mpimg.imread(path)
plt.imshow(im)
img = image.load_img(path,target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
img_preprocessed = tf.keras.applications.vgg16.preprocess_input(img)
prediction = model.predict(img_preprocessed)
print (prediction)
if prediction[0][1] < 0.5:
    print("With Mask")

else:
    print("Without Mask")